# Analyzing Orangetheory's Customer Sentiment on Lift45 and HR Metrics

## Setup for Extracting Reddit posts and comments

### Importing relevant packages

In [1]:
import praw
import time
import json
from datetime import datetime, timedelta
import codecs
import os
from dotenv import load_dotenv

### Initializing a Reddit object and defining the subreddit

In [2]:
load_dotenv()

reddit = praw.Reddit(
   client_id=os.getenv("REDDIT_CLIENT_ID"),
   client_secret=os.getenv("REDDIT_CLIENT_SECRET"),
   user_agent=os.getenv("REDDIT_USER_AGENT"),
   username=os.getenv("REDDIT_USERNAME"),
   password=os.getenv("REDDIT_PASSWORD")
)

subreddit = reddit.subreddit("orangetheory")

## Fetching all Reddit posts, comments, and replies about Lift45 from last week

In [43]:
keywords = ['lift45', 'lift 45']
one_week_ago = int(time.time()) - 7 * 24 * 60 * 60

In [44]:
def fetch_comments(comment, posts):
    # Add comment to posts
    decoded_comment = codecs.unicode_escape_decode(comment.body)[0]
    clean_comment = decoded_comment.replace('’', "'")
    clean_comment = clean_comment.encode('ascii', 'ignore').decode()
    
    posts.append({
        'date': datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d'),
        'author': comment.author.name if comment.author else 'Deleted',
        'title': f'Re: {post.title}',
        'body': clean_comment.replace('\n', ' ').replace('\t', ' ').replace('\r', ' ')
    })

    # Process replies to the comment
    for reply in comment.replies:
        fetch_comments(reply, posts)

posts = []

for post in subreddit.new(limit = 1000):
    if any(keyword in post.title.lower() for keyword in keywords):
        if post.created_utc >= one_week_ago:
            decoded_text = codecs.unicode_escape_decode(post.selftext)[0]
            clean_text = decoded_text.replace('’', "'")
            clean_text = clean_text.encode('ascii', 'ignore').decode()
            
            posts.append({
                'date': datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d'),
                'author': post.author.name if post.author else 'Deleted',
                'title': post.title,
                'body': clean_text.replace('\n', ' ').replace('\t', ' ').replace('\r', ' ')
            })
            
            for comment in post.comments:
                fetch_comments(comment, posts)

/var/folders/nl/4h1npm454flgnps4vlz6y_700000gn/T/ipykernel_54082/3147963917.py:3: DeprecationWarning: invalid escape sequence '\~'
  decoded_comment = codecs.unicode_escape_decode(comment.body)[0]


In [45]:
len(posts)

81

In [48]:
posts[0]

{'date': '2023-07-22',
 'author': 'moonrivervoyages',
 'title': 'Regular class into Lift45 in place of a 90 minute?',
 'body': 'The studio I go to each Saturday for their 90 minute class has taken them off the schedule. My home studio has a regular 7 am class and then a Lift45 at 8:15. Have any of you ever tried doing this? I feel like it would be doable because I am used to doing a 90 once a week but wanted to know what others think!'}

## ChatGPT API Setup
### Load the API key and relevant Python libaries.
In this course, we've provided some code that loads the OpenAI API key for you.

In [7]:
import os
import openai
import sys
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.environ['OPENAI_API_KEY']

In [8]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

## Determine categories of feedback based on last week's posts and comments

In [46]:
# Combine all post bodies into one string
all_posts = " ###END_OF_COMMENT### ".join([post['body'] for post in posts])

In [47]:
all_posts

'The studio I go to each Saturday for their 90 minute class has taken them off the schedule. My home studio has a regular 7 am class and then a Lift45 at 8:15. Have any of you ever tried doing this? I feel like it would be doable because I am used to doing a 90 once a week but wanted to know what others think! ###END_OF_COMMENT### I prefer to do the Lift45 before the regular 60. But yes its doable. ###END_OF_COMMENT### Oooo yes smart idea however that isn\'t an option for me because of my work schedule! ###END_OF_COMMENT### Just know if you have a limited number of classes going the Lift + 60 will count as two classes and doing the 90 only counts as one. ###END_OF_COMMENT### Thank you so much! Good to know! ###END_OF_COMMENT### I found some information that could be relevant to your question or topic.      Take a look at [previous conversations about pairing Lift 45 with a regular class](https://www.google.com/search?q=regular+60+lift45+site:reddit.com/r/orangetheory)      _This is an 

In [56]:
prompt = f"""
Your role is to analyze and summarize the main topics of customer sentiment and feedback about the Lift45 \
course from a series of social media posts and comments. Lift45 is a course offered by the fitness company \
Orangetheory, sometimes referred to as OTF. 

You are presented with a compilation of these posts and comments, delimited by triple backticks. 

Analyze the text and identify the top topics that customers are positively and negatively commenting on. \
Focus only on the specific compliments, criticisms, and feedback about Lift45. Ignore any unrelated comments. \
* Use at most 4 words as the topic name and give a brief explanation of each. 
* Do not use snacks as one of the topics. 
* Give me 5 positive topics and 5 negative topics.

Social Media Posts: ```{all_posts}```
"""

response = get_completion(prompt)
print(response)

Positive topics:
1. Lift45 + Regular Class: Customers express that it is doable and enjoyable to pair Lift45 with a regular 60-minute class. They appreciate the opportunity to do both workouts and find it beneficial for their fitness goals.
2. Cardio Between Lifting: Customers mention that they enjoy the cardio exercises between lifting sets during Lift45. They find it effective and enjoyable for their workout.
3. Trainer's Opinion: Customers appreciate hearing from a NASM certified trainer who suggests doing the regular class before Lift45 for fat loss and vice versa for strength. They value the professional advice and find it helpful in planning their workouts.
4. Challenging Templates: Customers express their excitement and satisfaction with the challenging Lift45 templates. They appreciate the opportunity to lift heavy and feel the burn during the workout.
5. Slow and Heavy: Customers enjoy Lift45 templates that emphasize slow and heavy lifting. They find it effective and enjoyable

### Original list of 10 categories
1. **pairing with regulars**: Customers express that it is doable and enjoyable to pair Lift45 with a regular 60-minute class. They appreciate the opportunity to do both workouts and find it beneficial for their fitness goals.

2. **cardio**: Customers mention that they enjoy the cardio exercises between lifting sets during Lift45. They find it effective and enjoyable for their workout.

3. **coaching**: Customers appreciate the coaches moving around and correcting form during the class. Customers appreciate hearing from a NASM certified trainer who suggests doing the regular class before Lift45 for fat loss and vice versa for strength. They value the professional advice and find it helpful in planning their workouts.

4. **intensity**: Customers express their excitement and satisfaction with the challenging Lift45 templates. They appreciate the opportunity to lift heavy and feel the burn during the workout.

5. **slow lifting**: Customers enjoy Lift45 templates that emphasize slow and heavy lifting. They find it effective and enjoyable for their workout.

6. **longer blocks**: Customers like the longer blocks and minimal switching between exercises, as it allows them to focus and go at their own pace.

7. **limited availability**: Customers mention that the schedule for Lift45 classes does not always align with their availability. They express frustration with long waitlists and difficulty incorporating Lift45 into their schedule.

8. **exercise dislikes**: Customers express dislike for certain exercises such as the pop squats in the Lift45 template. They find it challenging and uncomfortable for their knees.

9. **repetitive templates**: Customers express a desire for more variety in the Lift45 templates. For example, they have done hip bridge exercises multiple times in a short period.

10. **lacking continuity**: Some customers mention that they prefer a 90-minute class over Lift45 because of the continuity it offers. They feel that the breaks between exercises in Lift45 disrupt the flow of their workout.

### Narrowing down categories that are too similar:

**exercise content**
* repetitive exercise and dislikes: Customers express a desire for more variety in the Lift45 templates. For example, they have done hip bridge exercises multiple times in a short period. Customers also express dislike for certain exercises. For example, they find the pop squats challenging and uncomfortable for their knees.
* cardio: Customers mention that they enjoy the cardio exercises between lifting sets during Lift45. They find it effective and enjoyable for their workout.
* intensity: Customers express their excitement and satisfaction with the challenging Lift45 templates. They appreciate the opportunity to lift heavy and feel the burn during the workout.
* longer blocks: Customers like the longer blocks and minimal switching between exercises, as it allows them to focus and go at their own pace. Customers enjoy Lift45 templates that emphasize slow and heavy lifting. They find it effective and enjoyable for their workout.
* lacking continuity: Some customers mention that they prefer a 90-minute class over Lift45 because of the continuity it offers. They feel that the breaks between exercises in Lift45 disrupt the flow of their workout.

**course relationship**
* How should a customer choose what classes to take and in what order. Some courses that are offered include Lift45, regular classes which are 60 minutes, and 90-minute classes.

**course availability**
* Customers mention that the schedule for Lift45 classes does not always align with their availability. They express frustration with long waitlists and difficulty incorporating Lift45 into their schedule.
    
**coaching**
* Customers appreciate the coaches moving around and correcting form during the class. Customers appreciate hearing from a NASM certified trainer who suggests doing the regular class before Lift45 for fat loss and vice versa for strength. They value the professional advice and find it helpful in planning their workouts.

**perceived price value**
* how course credits are used and counted

## Writing a prompt for creating a list of posts and comments that are about customer sentiment of Lift45

### Using the first 10 posts and comments for testing the prompt

In [71]:
test_posts = posts[:10]

In [107]:
test_posts

[{'date': '2023-07-22',
  'author': 'moonrivervoyages',
  'title': 'Regular class into Lift45 in place of a 90 minute?',
  'body': 'The studio I go to each Saturday for their 90 minute class has taken them off the schedule. My home studio has a regular 7 am class and then a Lift45 at 8:15. Have any of you ever tried doing this? I feel like it would be doable because I am used to doing a 90 once a week but wanted to know what others think!'},
 {'date': '2023-07-22',
  'author': 'runningJD_82',
  'title': 'Re: Regular class into Lift45 in place of a 90 minute?',
  'body': 'I prefer to do the Lift45 before the regular 60. But yes its doable.'},
 {'date': '2023-07-22',
  'author': 'moonrivervoyages',
  'title': 'Re: Regular class into Lift45 in place of a 90 minute?',
  'body': "Oooo yes smart idea however that isn't an option for me because of my work schedule!"},
 {'date': '2023-07-22',
  'author': 'TuesGirl',
  'title': 'Re: Regular class into Lift45 in place of a 90 minute?',
  'body':

<!-- Example post for posts that do not contain customer sentiment, compliments, criticisms, and feedback:
    'date': '2023-07-22',
    'author': 'moonrivervoyages',
    'title': 'Regular class into Lift45 in place of a 90 minute?',
    'body': 'The studio I go to each Saturday for their 90 minute class has taken them off the schedule. \
     My home studio has a regular 7 am class and then a Lift45 at 8:15. Have any of you ever tried doing this? \
     I feel like it would be doable because I am used to doing a 90 once a week but wanted to know what \
     others think!'
     
    Example response for above post: 'Irrelevant feedback.' -->

### Prompt Template

In [105]:
json_objects = []

for i in range(len(test_posts)):
    prompt = f"""
    Your role is to summarize relevant customer sentiment information from a social media post or comment about \
    the course Lift45, which is offered by the fitness company Orangetheory. Orangetheory is sometimes referred \
    to as OTF. 

    The social media post or comment of interest is delimited by 3 backticks. 
    Social Media Post: ```{posts[i]['body']}```

    The below 10 fields are the JSON keys that you should output after analyzing the post. 

    1. date: #{test_posts[i]['date']}#
    
    2. topic: 'Lift45'

    3. title: ##{test_posts[i]['title']}##

    4. source: 'Reddit'

    5. author: ###{test_posts[i]['author']}###

    6. sentiment: Determine whether the post/comment has a positive, neutural, or negative customer sentiment. \
    Use 1 of the 3 words only.
    
    7. summary: Provide a 1 sentence summary on the post, focusing on customer sentiment and/or compliments, \
    criticisms, and feedback about Lift45.
     
    8. text: Only extract text that is about customer sentiment and/or compliments, criticisms, \
    and feedback about Lift45. Do not extract any other texts.   

    9. category: 
    
        5 categories of feedback are as follows:

        - exercise content: How effective are the exercrises in Lift45 in terms of activity type, \
        intensity, and pace. 
        
        - course relationship: How should a customer choose what classes to take and in what order. \ Some \
        courses that are offered include Lift45, regular classes which are 60 minutes, and 90-minute classes.

        - course availability: How does scheduling and capacity constraints affect customers' ability to \
        join Lift45 classes.

        - coaching: How professional and helpful the trainer is.

        - perceived price value: how course credits are used and counted.
        
    Categorize the post/comment as one of the 5 above listed categories that the author is providing feedback \
    about, including 'exercise content', course relationship', 'course availability', 'coaching', \
    and 'perceived price value'. Use "other" if the post/comment does not fall into any of the 5 categories. \
    Do not leave blank.
    
    ----------
    YOUR TASK: 
    Analyze the post for each of the JSON keys mentioned above and \
    output the result in JSON format: date, topic, title, source, author, sentiment, summary, text, and area.
    
    """

    response = get_completion(prompt)
    print(i, response, "\n")
    
    # Load the response string as a dictionary and add to the list
    json_objects.append(json.loads(response))

    time.sleep(20)

0 {
    "date": "2023-07-22",
    "topic": "Lift45",
    "title": "Regular class into Lift45 in place of a 90 minute?",
    "source": "Reddit",
    "author": "moonrivervoyages",
    "sentiment": "neutral",
    "summary": "The author is seeking opinions on whether it is doable to replace a 90-minute class with a Lift45 class and wants to know what others think.",
    "text": "Have any of you ever tried doing this? I feel like it would be doable because I am used to doing a 90 once a week but wanted to know what others think!",
    "category": "course relationship"
} 

1 {
    "date": "2023-07-22",
    "topic": "Lift45",
    "title": "Re: Regular class into Lift45 in place of a 90 minute?",
    "source": "Reddit",
    "author": "runningJD_82",
    "sentiment": "positive",
    "summary": "The author prefers to do Lift45 before the regular 60-minute class and finds it doable.",
    "text": "I prefer to do the Lift45 before the regular 60. But yes its doable.",
    "category": "course relat

## Fetching all Reddit posts, comments, and replies about Lift45 from the last 3 months